## Demo_Match

Abstract:
- Match image_file to folder with images

## Run name

In [1]:
import time

project_name = 'Google_LandMark_Rec'
step_name = 'Demo_Match'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google_LandMark_Rec_Demo_Match_20180510_193004


## Import PKGs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py
import cv2
from PIL import Image

from tqdm import tqdm
import multiprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


cpu_amount:  4


## Project folders

In [3]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## Preview csv

In [4]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...


In [5]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']
print('len(train_landmark_id) = \t%s' % len(list(set(train_landmark_id))))

id_2_landmark_id_dict = dict(zip(train_id, train_landmark_id))
print('len(id_2_landmark_id_dict) = \t%d' % len(id_2_landmark_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))

len(train_landmark_id) = 	14951
len(id_2_landmark_id_dict) = 	1225029
id: cacf8152e2d2ae60, 	landmark_id:4676
id: 0a58358a2afd3e4e, 	landmark_id:6651


## OrbMatch

In [6]:
val_images = os.listdir(val_folder)
print(len(val_images))
image_file = os.path.join(val_folder, val_images[0])
print(image_file)

60885
/data1/kaggle/landmark-recognition-challenge/input/data_val/69b846bd58c3f09a.jpg


In [23]:
%%time

# bf.match(des1, des2)

class OrbMatch(object):
    def __init__(
        self,
        image_file, 
        original_folder, 
        n_features=100, 
        is_crossCheck = False, 
        n_matches=100, 
        min_distance=50, 
        min_good_match=30,
        n_class = 14951,
        top=3,
    ):
        self._image_file = image_file
        self._original_folder = original_folder
        
        self._n_features = n_features
        self._is_crossCheck = is_crossCheck
        self._n_matches = n_matches
        self._min_distance = min_distance
        self._min_good_match = min_good_match
        self._n_class = n_class
        self._top = top
        
        self._key_point = None
        self._destance = None
        
        self._clf = cv2.ORB_create(self._n_features)
        self._bf  = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=self._is_crossCheck)
#         self._class_weight = class_weight
        
        print(self._image_file)
    
    def get_class_weight(self):
        pass
        
    def image_detect_and_compute(self, image_file):
        """Detect and compute interest points and their descriptors."""
        img = cv2.imread(image_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = self._clf.detectAndCompute(img, None)
        return des
        
    def match(self):
        img = cv2.imread(self._image_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        des1 = self.image_detect_and_compute(self._image_file)
        matched_image_classes = np.zeros(self._n_class)
        
        image_names = os.listdir(self._original_folder)
        rows = len(image_names)
        rows = 100
        print(len(image_names[:rows]))
        for i, file_name in enumerate(image_names[:rows]):
            image_id = file_name[:-4]
            image_file = os.path.join(self._original_folder, file_name)
#             print(image_file, end=' ')
            des2 = self.image_detect_and_compute(image_file)
            try:
                matches = self._bf.match(des1, des2)
            except Exception as ex:
                print(image_file)
                print(ex)
                continue
#             matches = sorted(matches, key = lambda x: x.distance) # Sort matches by distance.  Best come first.
            matches = list(filter(lambda x: x.distance < self._min_distance, matches))
            if len(matches) > self._min_good_match:
                class_idx = id_2_landmark_id_dict[image_id]
                matched_image_classes[class_idx] = matched_image_classes[class_idx] + 1
                print('len(matches)=', len(matches), end=' ')
                print('+%s' % class_idx, end=' ')
#             else:
#                 print('-', end='')
                
            if i % 10 == 0:
                print('%s' % int(i/10), end=' ')
#         matched_image_classes = matched_image_classes / sum(matched_image_classes)

        print('')
        sorted_matches_idx = np.argsort(matched_image_classes)
        sorted_matches_idx = sorted_matches_idx[::-1]
        best_matches_idx = sorted_matches_idx[0:self._top]
        best_matches = {}
        for i in best_matches_idx:
            best_matches[i] = matched_image_classes[i]
        return best_matches


orb_match = OrbMatch(image_file, val_folder)
best_matches = orb_match.match()
print('len(best_matches) = ', len(best_matches))
print(best_matches)
print('*'*80)

orb_match = OrbMatch(image_file, val_folder, 100, True)
best_matches = orb_match.match()
print('len(best_matches) = ', len(best_matches))
print(best_matches)
print('*'*80)

/data1/kaggle/landmark-recognition-challenge/input/data_val/69b846bd58c3f09a.jpg
100
len(matches)= 100 +6051 0 1 2 3 4 5 6 7 8 9 
len(best_matches) =  3
{6051: 1.0, 14950: 0.0, 4993: 0.0}
********************************************************************************
/data1/kaggle/landmark-recognition-challenge/input/data_val/69b846bd58c3f09a.jpg
100
len(matches)= 100 +6051 0 1 2 3 4 5 6 7 8 9 
len(best_matches) =  3
{6051: 1.0, 14950: 0.0, 4993: 0.0}
********************************************************************************
CPU times: user 9.33 s, sys: 200 ms, total: 9.53 s
Wall time: 8.28 s


In [27]:
%%time

# bf.match(des1, des2)

class OrbMatch(object):
    def __init__(
        self,
        image_file, 
        original_folder, 
        n_features=100, 
        is_crossCheck = False, 
        n_matches=100, 
        min_distance=50, 
        min_good_match=30,
        n_class = 14951,
        top=3,
    ):
        self._image_file = image_file
        self._original_folder = original_folder
        
        self._n_features = n_features
        self._is_crossCheck = is_crossCheck
        self._n_matches = n_matches
        self._min_distance = min_distance
        self._min_good_match = min_good_match
        self._n_class = n_class
        self._top = top
        
        self._key_point = None
        self._destance = None
        
        self._clf = cv2.ORB_create(self._n_features)
        self._bf  = cv2.BFMatcher(cv2.NORM_L2, crossCheck=self._is_crossCheck)
#         self._class_weight = class_weight
        
        print(self._image_file)
    
    def get_class_weight(self):
        pass
        
    def image_detect_and_compute(self, image_file):
        """Detect and compute interest points and their descriptors."""
        img = cv2.imread(image_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = self._clf.detectAndCompute(img, None)
        return des
        
    def match(self):
        img = cv2.imread(self._image_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        des1 = self.image_detect_and_compute(self._image_file)
        matched_image_classes = np.zeros(self._n_class)
        
        image_names = os.listdir(self._original_folder)
        rows = len(image_names)
        rows = 100
        print(len(image_names[:rows]))
        for i, file_name in enumerate(image_names[:rows]):
            image_id = file_name[:-4]
            image_file = os.path.join(self._original_folder, file_name)
#             print(image_file, end=' ')
            des2 = self.image_detect_and_compute(image_file)
            try:
                matches = self._bf.knnMatch(des1, des2, k=3)
            except Exception as ex:
                print(image_file)
                print(ex)
                continue
#             matches = sorted(matches, key = lambda x: x.distance) # Sort matches by distance.  Best come first.
            matches = list(filter(lambda x: x.distance < self._min_distance, matches))
            if len(matches) > self._min_good_match:
                class_idx = id_2_landmark_id_dict[image_id]
                matched_image_classes[class_idx] = matched_image_classes[class_idx] + 1
                print('len(matches)=', len(matches), end=' ')
                print('+%s' % class_idx, end=' ')
#             else:
#                 print('-', end='')
                
            if i % 10 == 0:
                print('%s' % int(i/10), end=' ')
#         matched_image_classes = matched_image_classes / sum(matched_image_classes)

        print('')
        sorted_matches_idx = np.argsort(matched_image_classes)
        sorted_matches_idx = sorted_matches_idx[::-1]
        best_matches_idx = sorted_matches_idx[0:self._top]
        best_matches = {}
        for i in best_matches_idx:
            best_matches[i] = matched_image_classes[i]
        return best_matches


# orb_match = OrbMatch(image_file, val_folder)
# best_matches = orb_match.match()
# print('len(best_matches) = ', len(best_matches))
# print(best_matches)
# print('*'*80)

orb_match = OrbMatch(image_file, val_folder, 100, True)
best_matches = orb_match.match()
print('len(best_matches) = ', len(best_matches))
print(best_matches)
print('*'*80)

/data1/kaggle/landmark-recognition-challenge/input/data_val/69b846bd58c3f09a.jpg
100
/data1/kaggle/landmark-recognition-challenge/input/data_val/69b846bd58c3f09a.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/19a1de4f08cd0305.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/4ee821754ef5fd83.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/5f7f65b33582323e.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/c1943ca95701cbc7.jpg
/io/opencv/modules/cor

/data1/kaggle/landmark-recognition-challenge/input/data_val/f02634b4e3eccb8a.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/a0a72fecb96757c3.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/1c20ea4f001b38e9.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/fb5e6fcf95f7ad91.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/54ab0fc30b09c78a.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function 

/data1/kaggle/landmark-recognition-challenge/input/data_val/39c1ec2b91dfef05.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/c70dc659b51bb960.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/652d7b9570e8d49d.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/65b4dd953adf4ace.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function batchDistance

/data1/kaggle/landmark-recognition-challenge/input/data_val/28b78609c6134219.jpg
/io/opencv/modules/core/src/stat.cpp:4049: error: (-215) K == 1 && update == 0 && mask.empty() in function 